In [65]:
# import libraries
import pandas as pd
import requests
import xml.dom.minidom as m
import xml.etree.ElementTree as et
import time as time

In [213]:
# make request using get to obtain 1000 Alzheimer (al) id
r_al_ids = requests.get("https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=Alzheimers+AND+2019[pdat]&retmax=1000&retmode=xml")

# parse returned response
al_doc = m.parseString(r_al_ids.text)
al_idlist = al_doc.getElementsByTagName("Id")

# adding ids to id list
al_ids = []
i = 0
while i < 1000:
    al_ids.append(al_idlist[i].childNodes[0].wholeText)
    i += 1


In [195]:
# get articles identified by ids through POST
# create dictionary with parameters for POST
al_param = {'db': 'pubmed', 'retmode': 'xml', 'id' : al_ids}
r_al = requests.post("https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi", data = al_param)
print(r_al.status_code)

200


In [228]:
# read metadata using ElementTree
# and save information from post request as json file
al_root = et.fromstring(r_al.text)
print("size of tree is:", len(al_root))
# print(al_root[0].tag, al_root[2].attrib)

for node in al_root: print(node.tag, node.attrib)

size of tree is: 1000
PubmedBookArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
Pubme

In [227]:
# col_name = ["title", "abstract", "query", "mesh"]

In [215]:
# make request using get to obtain 1000 cancer (cn) id
r_cn_ids = requests.get("https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=cancer+AND+2019[pdat]&retmax=1000&retmode=xml")

# parse returned response
cn_doc = m.parseString(r_cn_ids.text)
cn_idlist = cn_doc.getElementsByTagName("Id")

# adding ids to id list
cn_ids = []
i = 0
while i < 1000:
    cn_ids.append(cn_idlist[i].childNodes[0].wholeText)
    i += 1

In [222]:
# get articles identified by ids through POST
# create dictionary with parameters for POST

cn_param = {'db': 'pubmed', 'retmode': 'xml', 'id': cn_ids}
r_cn = requests.post("https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi", data = cn_param)

print(r_cn.status_code)

200


In [226]:
# read metadata using ElementTree
# save information from post request as json file
cn_root = et.fromstring(r_cn.text)
print("size of tree is:", len(cn_root))
for node in cn_root: print(node.tag, node.attrib)

size of tree is: 1000
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedBookArticle {}
PubmedBookArticle {}
PubmedBookArticle {}
PubmedBookArticle {}
PubmedBookArticle {}
PubmedBookArticle {}
PubmedBookArticle {}
PubmedBookArticle {}
PubmedBookArticle {}
PubmedBookArticle {}
PubmedBookArticle {}
PubmedBookArticle {}
PubmedBookArticle {}
PubmedBookArticle {}
PubmedBookArticle {}
PubmedBookArticle {}
PubmedBookArticle {}
PubmedBookArticle {}
PubmedBookArticle {}
PubmedBookArticle {}
PubmedBookArticle {}
PubmedBookArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedArticle {}
PubmedBookArticle {}
PubmedBookArticle {}
PubmedBookArticle {}
PubmedBookA